# FEM Modelling for Three-Phase STEDIN Transformer

### Voltage-fed Couple Circuit Analysis

In frequency domain,
\begin{equation}
    -\nabla\cdot\Big[\frac{1}{\mu}\nabla u_z \Big] + j\omega\sigma u_z = J_0 = \frac{NI}{S}
\end{equation}
where,

$N$ = number of turns,

$S$ = cross-sectional area.

Circuit equation for an R-L equivalent network,
\begin{align}
    V & = j\omega N \psi + R_{ext}I + j\omega L_{ext} I \\
    V & = j\omega G'u + R_{ext}I + j\omega L_{ext} I
\end{align}

Combining the diffusion equation and circuit equation, 
\begin{equation}
    \begin{bmatrix}
        A & -f \\
        j\omega G^T & R_{ext} + j\omega L_{ext}
    \end{bmatrix} \begin{bmatrix}
        u \\
        I
    \end{bmatrix} = \begin{bmatrix}
        0 \\
        V
    \end{bmatrix}
\end{equation}
where, $V$ = known voltages, and

\begin{equation}
    G = f = \frac{N}{S}\times \frac{area\ of\ element}{3} \begin{bmatrix}
    1 \\ 1 \\ 1
\end{bmatrix}
\end{equation}

Note : The above formulation is only valid in case of per unit z-direction axis.

## Import Packages

In [1]:
print(" ▸ Loading Packages .... \r")
start = time_ns()
using gmsh
using LinearAlgebra
using LaTeXStrings
using Plots
using CSV
using DataFrames
include("../../../../lib/Makie_Plots.jl");
using .Makie_Plots
include("../../../../lib/FEM_VoltageFed_Tri_1e.jl");
using .FEM_VoltageFed_Tri_1e
include("../../../../lib/Post_Process_Frequency.jl");
using .Post_Process_Frequency
include("../../../../lib/Mesh_Data_stedin.jl");
using .Mesh_Data_stedin
include("../../../../lib/Save_VTK.jl");
using .Save_VTK
elapsed = round((time_ns() - start)/10^9, digits=2)
println(" ✓ Packages loaded ("*string(elapsed)*" seconds)                               ")

 ✓ Packages loaded (23.24 seconds)                               


## Julia code

In [2]:
gmsh.initialize()

## Read mesh from file
print(" ▸ Reading mesh file .... \r")
start = time_ns()
gmsh.open("../../../../Geometry/mesh/stedin_transformer.msh")
elapsed = round((time_ns() - start)/10^9, digits=2)
println(" ✓ Mesh file loaded ("*string(elapsed)*" seconds)                               ")

print(" ▸ Generating required mesh data .... \r")
start = time_ns()
mesh_data = get_mesh_data_tri_1e(gmsh)
elapsed = round((time_ns() - start)/10^9, digits=2)
println(" ✓ Mesh data generated ("*string(elapsed)*" seconds)                               ")

print(" ▸ Setting initial parameters .... \r")
start = time_ns()
S = 400e3;                    # Power rating
Vp = 10750 * sqrt(2);         # Primary peak phase voltage
Vs = 420 * sqrt(2/3);         # Secondary peak phase voltage
Ip = (S/10750) * sqrt(2/9);   # Primary peak phase current
Is = (S/420) * sqrt(2/3);     # Secondary peak phase current
Np = 266;                     # Primary turns
Ns = 6;                       # Secondary turns

# External resistance Rext
Rp = 1.8131 + 600
Rs = 1.2999e-3 + 0.2
CFFp = 0.3
CFFs = 0.3

# External Inductance Lext
Lext = 1e-6

# Z-axis length
z_len_p = 0.4
z_len_s = 0.4

omega = 2*pi*50;  # Frequency

# HV winding dimensions (all phases left/right are identical)
wwhv = 3e-2;
hwhv = 74e-2;
mwhv = 14.75e-2;
Awhv = wwhv * hwhv;

# LV winding dimensions (all phases left/right are identical)
wwlv = 2e-2;
hwlv = 74e-2;
mwlv = 11.25e-2;
Awlv = wwlv * hwlv;

# Calculate turn density in the windings
Tp = Np / Awhv;
Ts = Ns / Awlv;

# Relative permeability model
mu0 = 4e-7 * pi;
mur = 2500;       # Relative permeability of the core
reluctivityfunction(group_id) = (1 / mu0) + (1/(mu0*mur) - 1/mu0) * (group_id == 2)
reluctivityperelement = map(reluctivityfunction, mesh_data.e_group);

# Conductivity
conductivityfunction(group_id) = 0;
conductivityperelement = map(conductivityfunction, mesh_data.e_group);

# Handle the boundary conditions
bnd_node_ids, _ = gmsh.model.mesh.getNodesForPhysicalGroup(1, 1);

elapsed = round((time_ns() - start)/10^9, digits=2)
println(" ✓ Initial parameters set ("*string(elapsed)*" seconds)                               ")

Info    : Reading '../../../../Geometry/mesh/stedin_transformer.msh'...
Info    : 168 entities
Info    : 10385 nodes
Info    : 20768 elements
Info    : Done reading '../../../../Geometry/mesh/stedin_transformer.msh'
 ✓ Mesh file loaded (0.06 seconds)                               
 ✓ Mesh data generated (3.81 seconds)                               
 ✓ Initial parameters set (0.09 seconds)                               


### Open Circuit Test (Core Loss)

In [3]:
print(" ▸ Setting initial parameters .... \r")
start = time_ns()
# Z-axis length
z_length = [z_len_s; z_len_s; z_len_s];

# Source turn density T
# One term for each of the windings, with a positive and negative part
sourcefunction(group_id) = [Ts*(-1*(group_id==9) + 1*(group_id==10)),
                            Ts*(-1*(group_id==11) + 1*(group_id==12)),
                            Ts*(-1*(group_id==13) + 1*(group_id==14))];
sourceperelement = map(sourcefunction, mesh_data.e_group);

# Source voltage V
# One term for each of the windings, with a positive and negative part
coil_voltage = [Vs*exp(1im*-2pi/3); Vs; Vs*exp(1im*2pi/3)];

# External resistance Rext
# One term for each of the windings, with a positive and negative part
ext_resistance = [Rs/CFFs; Rs/CFFs; Rs/CFFs];

# External inductance Lext
# One term for each of the windings, with a positive and negative part
ext_inductance = Lext .* [1; 1; 1];
elapsed = round((time_ns() - start)/10^9, digits=2)
println(" ✓ Initial parameters set ("*string(elapsed)*" seconds)                               ")

# Calculate the vector potential
u = fem(mesh_data, sourceperelement, reluctivityperelement, conductivityperelement, omega, bnd_node_ids, coil_voltage, ext_resistance, ext_inductance, z_length);

# Post-process for magnetic field and current density
Bx, By, B, Hx, Hy, H, mag_energy = post_process(mesh_data, u, reluctivityperelement);

# Calculate source current density
Jel = source_current_density(mesh_data, u, sourceperelement);

# Calculate core loss
Pv, Pcore = core_loss(mesh_data, B, z_length);
println("Core loss = "*string(Pcore)*" W")

print(" ▸ Saving Plots .... \r")
start = time_ns()
# Contour plot of the magnetic flux density
plot_surface_tri_1e(norm.(B), "stedin_trafo_B_voltagefed_OC", gmsh)
# Contour plot of the magnetic field strength
plot_surface_tri_1e(norm.(H), "stedin_trafo_H_voltagefed_OC", gmsh)
# Contour plot of the magnetic energy
plot_surface_tri_1e(norm.(mag_energy), "stedin_trafo_energy_voltagefed_OC", gmsh)
# Contour plot of the relative permeability
plot_surface_tri_1e((1 ./reluctivityperelement ./mu0), "stedin_trafo_mu_voltagefed_OC", gmsh)
# Contour plot of the source current density
plot_surface_tri_1e(norm.(Jel), "stedin_trafo_Jel_voltagefed_OC", gmsh)
# Contour plot of the Pv
plot_surface_tri_1e(norm.(Pv), "stedin_trafo_Pv_voltagefed_OC", gmsh)
elapsed = round((time_ns() - start)/10^9, digits=2)
println(" ✓ Plots saved ("*string(elapsed)*" seconds)                               ")

print(" ▸ Saving VTK file .... \r")
start = time_ns()
# Save as VTK file for Paraview visualization
save_vtk(mesh_data, imag(u[1:mesh_data.nnodes]), norm.(B), norm.(H), norm.(mag_energy), reluctivityperelement, norm.(Jel), norm.(Pv), "stedin_trafo_voltagefed_OC")
elapsed = round((time_ns() - start)/10^9, digits=2)
println(" ✓ VTK file saved ("*string(elapsed)*" seconds)                               ")

 ✓ Initial parameters set (0.17 seconds)                               
 ✓ Matrices constructed (0.05 seconds)                               
 ✓ Solution computed (0.86 seconds)                               
 ✓ Post processing variables computed (0.01 seconds)                               
 ✓ Current density computed (0.07 seconds)                               
 ✓ Core loss computed (0.0 seconds)                               
Core loss = 939.1115515057043 W
 ✓ Plots saved (19.16 seconds)                               
 ✓ VTK file saved (1.27 seconds)                               


#### STEDIN Transformer : Magnetic field density
<img src="../img/stedin_trafo_B_voltagefed_OC.png">

#### STEDIN Transformer : Magnetic field strength
<img src="../img/stedin_trafo_H_voltagefed_OC.png">

#### STEDIN Transformer : Magnetic energy
<img src="../img/stedin_trafo_energy_voltagefed_OC.png">

#### STEDIN Transformer : Relative permeability
<img src="../img/stedin_trafo_mu_voltagefed_oC.png">

#### STEDIN Transformer : Source Current Density
<img src="../img/stedin_trafo_Jel_voltagefed_OC.png">

#### STEDIN Transformer : Pv
<img src="../img/stedin_trafo_Pv_voltagefed_OC.png">

### Short Circuit Test (Winding Losses)

In [4]:
print(" ▸ Setting initial parameters .... \r")
start = time_ns()
# Z-axis length
z_length = [z_len_p; z_len_p; z_len_p;
            z_len_s; z_len_s; z_len_s];

# Source turn density T
# One term for each of the windings, with a positive and negative part
sourcefunction(group_id) = [Tp*(1*(group_id==3) - 1*(group_id==4)),
                            Tp*(1*(group_id==5) - 1*(group_id==6)),
                            Tp*(1*(group_id==7) - 1*(group_id==8)),
                            Ts*(-1*(group_id==9) + 1*(group_id==10)),
                            Ts*(-1*(group_id==11) + 1*(group_id==12)),
                            Ts*(-1*(group_id==13) + 1*(group_id==14))];
sourceperelement = map(sourcefunction, mesh_data.e_group);

# Source voltage V
# One term for each of the windings, with a positive and negative part
coil_voltage = [Vp*exp(1im*-2pi/3); Vp; Vp*exp(1im*2pi/3);
                0; 0; 0];

# External resistance Rext
# One term for each of the windings, with a positive and negative part
ext_resistance = [Rp/CFFp; Rp/CFFp; Rp/CFFp; 
                  Rs/CFFs; Rs/CFFs; Rs/CFFs];
    
# External resistance Rext
# One term for each of the windings, with a positive and negative part
ext_inductance = Lext .* [1; 1; 1; 
                          1; 1; 1];

elapsed = round((time_ns() - start)/10^9, digits=2)
println(" ✓ Initial parameters set ("*string(elapsed)*" seconds)                               ")

# Calculate the vector potential
u = fem(mesh_data, sourceperelement, reluctivityperelement, conductivityperelement, omega, bnd_node_ids, coil_voltage, ext_resistance, ext_inductance, z_length);

# Post-process for magnetic field and current density
Bx, By, B, Hx, Hy, H, mag_energy = post_process(mesh_data, u, reluctivityperelement);

# Calculate source current density
Jel = source_current_density(mesh_data, u, sourceperelement);

ext_resistance = [Rp; Rp; Rp; 
                  Rs; Rs; Rs];
# Calculate winding loss
Pwindingp, Pwindings = winding_loss(mesh_data, u, ext_resistance)
println("Primary winding loss = "*string(Pwindingp)*" W")
println("Secondary winding loss = "*string(Pwindings)*" W")

print(" ▸ Saving Plots .... \r")
start = time_ns()
# Contour plot of the magnetic flux density
plot_surface_tri_1e(norm.(B), "stedin_trafo_B_voltagefed_SC", gmsh)
# Contour plot of the magnetic field strength
plot_surface_tri_1e(norm.(H), "stedin_trafo_H_voltagefed_SC", gmsh)
# Contour plot of the magnetic energy
plot_surface_tri_1e(norm.(mag_energy), "stedin_trafo_energy_voltagefed_SC", gmsh)
# Contour plot of the relative permeability
plot_surface_tri_1e((1 ./reluctivityperelement ./mu0), "stedin_trafo_mu_voltagefed_SC", gmsh)
# Contour plot of the source current density
plot_surface_tri_1e(norm.(Jel), "stedin_trafo_Jel_voltagefed_SC", gmsh)
elapsed = round((time_ns() - start)/10^9, digits=2)
println(" ✓ Plots saved ("*string(elapsed)*" seconds)                               ")

print(" ▸ Saving VTK file .... \r")
start = time_ns()
# Save as VTK file for Paraview visualization
save_vtk(mesh_data, imag(u[1:mesh_data.nnodes]), norm.(B), norm.(H), norm.(mag_energy), reluctivityperelement, norm.(Jel), "stedin_trafo_voltagefed_SC")
elapsed = round((time_ns() - start)/10^9, digits=2)
println(" ✓ VTK file saved ("*string(elapsed)*" seconds)                               ")

gmsh.finalize()

 ✓ Initial parameters set (0.11 seconds)                               
 ✓ Matrices constructed (0.02 seconds)                               
 ✓ Solution computed (0.13 seconds)                               
 ✓ Post processing variables computed (0.01 seconds)                               
 ✓ Current density computed (0.12 seconds)                               
 ✓ Winding loss computed (0.13 seconds)                               
Primary winding loss = 10263.992332322858 W
Secondary winding loss = 11784.38293289698 W
 ✓ Plots saved (0.77 seconds)                               
 ✓ VTK file saved (0.1 seconds)                               


#### STEDIN Transformer : Magnetic field density
<img src="../img/stedin_trafo_B_voltagefed_SC.png">

#### STEDIN Transformer : Magnetic field strength
<img src="../img/stedin_trafo_H_voltagefed_SC.png">

#### STEDIN Transformer : Magnetic energy
<img src="../img/stedin_trafo_energy_voltagefed_SC.png">

#### STEDIN Transformer : Relative permeability
<img src="../img/stedin_trafo_mu_voltagefed_SC.png">

#### STEDIN Transformer : Source Current Density
<img src="../img/stedin_trafo_Jel_voltagefed_SC.png">